In [20]:
import openai
openai.api_key = OPENAI_API_KEY

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

emb = response['data'][0]['embedding']
print(len(emb))

1536


In [23]:
# print(emb)

In [24]:
## REMEMBER TO DELETE THIS LINE BEFORE UPLOADING
api_key = COHERE_API_KEY

In [25]:
import cohere  
co = cohere.Client(api_key)  
texts = [  
   'Hello from Cohere!'
]  
response = co.embed(texts=texts, model='embed-english-light-v2.0')  
embeddings = response.embeddings # All text embeddings 
print(len(embeddings[0])) # Print embeddings for the first text

1024


In [26]:
import pinecone      

pinecone.init(      
	api_key=PINECONE_API_KEY,      
	environment='us-west1-gcp'      
)      
index = pinecone.Index('pokemon-cards-v2')

/home/ec3dev/personal/github/pyenv/pte/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [27]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [29]:
pinecone.describe_index("pokemon-cards-v2")

IndexDescription(name='pokemon-cards-v2', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [30]:
from datasets import load_dataset

In [31]:
pcds = load_dataset("bhavnicksm/PokemonCardsPlus", split='train')
pcds

Generating train split: 100%|██████████| 13139/13139 [00:00<00:00, 185515.20 examples/s]


Dataset({
    features: ['id', 'name', 'card_image', 'pokemon_image', 'caption', 'pokemon_intro', 'pokedex_text', 'hp', 'set_name', 'blip_caption'],
    num_rows: 13139
})

In [32]:
pcds = pcds.map(lambda example: {"text" : f"Name: {example['name']}\nSet Name: {example['set_name']}\nPokemon Caption: {example['blip_caption']}\nCard Description: {example['caption']}\nPokemon Description: {example['pokemon_intro']}\nPokedex Entry: {example['pokedex_text']}"})

Map: 100%|██████████| 13139/13139 [00:01<00:00, 8815.29 examples/s]


In [33]:
pcds

Dataset({
    features: ['id', 'name', 'card_image', 'pokemon_image', 'caption', 'pokemon_intro', 'pokedex_text', 'hp', 'set_name', 'blip_caption', 'text'],
    num_rows: 13139
})

In [35]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

In [36]:
pcds = pcds.map(lambda example : {"length" : len(enc.encode(example['text']))})

Map: 100%|██████████| 13139/13139 [00:05<00:00, 2254.06 examples/s]


In [39]:
# It costs less than $1
sum(list(pcds['length'])) * (0.0001/1000)

0.8565002

In [77]:
pcds = pcds.map(lambda example : {"pokemon_image" : example['pokemon_image'] if example['pokemon_image'] != None else ''})
pcds = pcds.map(lambda example : {"pokemon_intro" : example['pokemon_intro'] if example['pokemon_intro'] != None else ''})
pcds = pcds.map(lambda example : {"pokedex_text" : example['pokedex_text'] if example['pokedex_text'] != None else ''})
pcds = pcds.map(lambda example : {"blip_caption" : example['blip_caption'] if example['blip_caption'] != None else ''})

Map: 100%|██████████| 13139/13139 [00:01<00:00, 9823.60 examples/s] 


In [78]:
from time import time, sleep

In [79]:
from torch.utils.data import DataLoader

In [87]:
dl = DataLoader(pcds, batch_size=64)

In [81]:
# emb = co.embed(texts=texts, model='embed-english-light-v2.0')

In [88]:
pinecone_obj = {'id': None, 'values': None, 'metadata' : None}

In [113]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13139}},
 'total_vector_count': 13139}

In [90]:
from tqdm import tqdm, trange

In [91]:
for batch in tqdm(dl):
    
    # print(batch)
    
    # text = batch['text']
    # response = openai.Embedding.create(
    # model="text-embedding-ada-002",
    # input=texts
    # )
    # emb = response['data'][0]['embedding']

    # upsert_obj = pinecone_obj.copy()

    # upsert_obj['id'] = batch['id']
    # upsert_obj['values'] = emb
    # upsert_obj['metadata'] = {"img_url": batch['card_image'],
    #                     "pokimg_url" : batch['pokemon_image'] if batch['pokemon_image'] != None else '', 
    #                     "name" : batch['name'], 
    #                     "description": batch['caption'],
    #                     "pokemon_intro": batch['pokemon_intro'] if batch['pokemon_intro'] != None else '', 
    #                     "pokedex_entry": batch['pokedex_text'] if batch['pokedex_text'] != None else '', 
    #                     "blip_caption" : batch['blip_caption'] if batch['blip_caption'] != None else '',  
    #                     "hp": int(batch['hp'])}
    
    # upsert_resp = index.upsert(vectors=[upsert_obj])


    upsert_list = []
    texts = batch['text']

    response = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=texts
    )

    # # print(len(response['data']))
    embs = [x['embedding'] for x in response['data']]
    # # print(len(embs))

    for i in range(len(batch['id'])):
        pcd = pinecone_obj.copy()
        pcd['id'] = batch['id'][i]
        pcd['values'] = embs[i]
        pcd['metadata'] = {"img_url": batch['card_image'][i],
                           "pokimg_url" : batch['pokemon_image'][i] if batch['pokemon_image'][i] != None else '', 
                           "name" : batch['name'][i], 
                           "description": batch['caption'][i],
                           "pokemon_intro": batch['pokemon_intro'][i] if batch['pokemon_intro'][i] != None else '', 
                           "pokedex_entry": batch['pokedex_text'][i] if batch['pokedex_text'][i] != None else '', 
                           "blip_caption" : batch['blip_caption'][i] if batch['blip_caption'][i] != None else '',  
                           "hp": int(batch['hp'][i])}
        upsert_list.append(pcd)
    
    # print(len(upsert_list))
    # print(upsert_list[0]['values'])
    
    upsert_resp = index.upsert(vectors=upsert_list)

    # break

100%|██████████| 206/206 [34:11<00:00,  9.96s/it]  


In [84]:
# index.delete(delete_all=True)

{}

In [92]:
query = "sad"

response = openai.Embedding.create(
                                model="text-embedding-ada-002",
                                input=query
                                )

query_emb = response['data'][0]['embedding']
query_emb

[0.00135776586830616,
 -0.01316700130701065,
 0.0049084085039794445,
 -0.026983262971043587,
 -0.01668599247932434,
 0.009440246038138866,
 -0.023009788244962692,
 -0.019283033907413483,
 0.00495385704562068,
 -0.03056718036532402,
 0.020049160346388817,
 0.02345128543674946,
 0.005989427212625742,
 -0.010738766752183437,
 -0.010764737613499165,
 0.024386221542954445,
 0.054641757160425186,
 -0.010173910297453403,
 0.025165332481265068,
 -0.027502670884132385,
 -0.004337059333920479,
 0.002937903394922614,
 -0.005103186704218388,
 -0.0020825027022510767,
 -0.0031164500396698713,
 -0.003684552852064371,
 0.01268005557358265,
 -0.04051385074853897,
 0.014101935550570488,
 -0.015010900795459747,
 0.016257479786872864,
 -0.009985624812543392,
 -0.0327746644616127,
 -0.017854660749435425,
 -0.022347543388605118,
 -0.016335392370820045,
 -0.03602096810936928,
 -0.020477673038840294,
 -0.0074080610647797585,
 -0.013387749902904034,
 0.006888652686029673,
 0.002709039021283388,
 0.006206929218

In [114]:
query_response = index.query(query_emb, top_k=5, include_metadata=True)
query_response

{'matches': [{'id': 'mcd21-24',
              'metadata': {'blip_caption': 'a pokemon that looks like a bug '
                                           'with a big nose',
                           'description': 'A Basic Pokemon Card of type Water '
                                          'with the title Sobble and 60 HP of '
                                          "rarity None from the set McDonald's "
                                          'Collection 2021 and the flavor '
                                          'text: When scared, this Pokemon '
                                          'cries. Its tears pack the chemical '
                                          'punch of 100 onions, and attackers '
                                          "won't be able to resist weeping. It "
                                          'has the attack Bind with the cost '
                                          'Water, Colorless, the energy cost 2 '
                                 

In [99]:
response_texts = [q['metadata']['description'] for q in query_response['matches']]
response_texts

["A Basic Pokemon Card of type Water with the title Sobble and 60 HP of rarity None from the set McDonald's Collection 2021 and the flavor text: When scared, this Pokemon cries. Its tears pack the chemical punch of 100 onions, and attackers won't be able to resist weeping. It has the attack Bind with the cost Water, Colorless, the energy cost 2 and the damage of 20 with the description: Flip a coin. If heads, your opponent's Active Pokemon is now Paralyzed. It has weakness against Lightning 2. ",
 "A Basic Pokemon Card of type Water with the title Sobble and 60 HP of rarity Promo from the set SWSH Black Star Promos and the flavor text: When scared, this Pokemon cries. Its tears pack the chemical punch of 100 onions, and attackers won't be able to resist weeping. It has the attack Bind with the cost Water, Colorless, the energy cost 2 and the damage of 20 with the description: Flip a coin. If heads, your opponent's Active Pokemon is now Paralyzed. It has weakness against Lightning 2. ",

In [121]:
coresponse = co.rerank(model="rerank-english-v2.0", query=query, documents=response_texts, top_n=5)
reranked_index = [coresponse[i].index for i in range(5)]
reranked_index

[1, 0, 2, 4, 3]

In [126]:
query_response['matches']

ValueError: 0 is not in list

In [127]:
reranked_responses = [query_response['matches'][ind] for ind in reranked_index]

In [128]:
reranked_responses

[{'id': 'swshp-SWSH003',
  'metadata': {'blip_caption': 'a pokemon that looks like a bug with a big nose',
               'description': 'A Basic Pokemon Card of type Water with the '
                              'title Sobble and 60 HP of rarity Promo from the '
                              'set SWSH Black Star Promos and the flavor text: '
                              'When scared, this Pokemon cries. Its tears pack '
                              'the chemical punch of 100 onions, and attackers '
                              "won't be able to resist weeping. It has the "
                              'attack Bind with the cost Water, Colorless, the '
                              'energy cost 2 and the damage of 20 with the '
                              'description: Flip a coin. If heads, your '
                              "opponent's Active Pokemon is now Paralyzed. It "
                              'has weakness against Lightning 2. ',
               'hp': 60.0,
        

In [115]:
qque = query_response['matches'].copy()
reranked_objs = []
for i in range(len(query_response)):
    text = 

5

In [84]:
qurls = [x['metadata']['img_url'] for x in query_response['matches']]
qurls[0]

'https://images.pokemontcg.io/swshp/SWSH003_hires.png'

In [85]:
from IPython.display import Image, display
from IPython.core.display import HTML 

In [86]:
import ipyplot

In [87]:
# for url in qurls:
#     display(Image(url=url, width=200))

ipyplot.plot_images(qurls, img_width=200)

# Using Qdrant

In [19]:
QDRANT_API_KEY = ...
OPENAI_API_KEY= ...

In [2]:
from datasets import load_dataset
pcds = load_dataset("bhavnicksm/PokemonCardsPlus", split='train')

/home/ec3dev/personal/github/pyenv/pte/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pcds = pcds.map(lambda example: {"text" : f"Name: {example['name']}\nSet Name: {example['set_name']}\nPokemon Caption: {example['blip_caption']}\nCard Description: {example['caption']}\nPokemon Description: {example['pokemon_intro']}\nPokedex Entry: {example['pokedex_text']}"})

In [4]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
pcds = pcds.map(lambda example : {"length" : len(enc.encode(example['text']))})
sum(list(pcds['length'])) * (0.0001/1000)

0.8565002

In [5]:
pcds = pcds.map(lambda example : {"pokemon_image" : example['pokemon_image'] if example['pokemon_image'] != None else ''})
pcds = pcds.map(lambda example : {"pokemon_intro" : example['pokemon_intro'] if example['pokemon_intro'] != None else ''})
pcds = pcds.map(lambda example : {"pokedex_text" : example['pokedex_text'] if example['pokedex_text'] != None else ''})
pcds = pcds.map(lambda example : {"blip_caption" : example['blip_caption'] if example['blip_caption'] != None else ''})

In [6]:
pcds = pcds.add_column(name='_id', column=[id for id in range(len(pcds))])

In [9]:
import openai
openai.api_key = OPENAI_API_KEY

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

emb = response['data'][0]['embedding']
print(len(emb))

1536


In [12]:
from tqdm import tqdm, trange

In [15]:
# put in the embeddings 

embslist = []

for obj in tqdm(pcds):
    response = openai.Embedding.create(
        model="text-embedding-ada-002", 
        input=obj['text']
    )
    emb = response['data'][0]['embedding']
    embslist.append(emb)

100%|██████████| 13139/13139 [1:35:09<00:00,  2.30it/s] 


In [16]:
len(embslist)

13139

In [17]:
pcds = pcds.add_column("embedding", embslist)
pcds[0]['embedding']

[-0.01602926477789879,
 0.002176301321014762,
 -0.023094359785318375,
 -0.002378825331106782,
 -0.04122190177440643,
 0.043479546904563904,
 0.0032354015856981277,
 -0.02632145956158638,
 0.007742387242615223,
 -0.03096954897046089,
 0.018738435581326485,
 0.010557800531387329,
 -0.022828754037618637,
 -0.012848644517362118,
 0.009873867966234684,
 0.021593689918518066,
 0.027994772419333458,
 -0.013253691606223583,
 0.006676646880805492,
 -0.0004168754385318607,
 0.027330759912729263,
 0.030199294909834862,
 -0.005156057886779308,
 -0.005531225353479385,
 0.008851287886500359,
 -0.0006532224360853434,
 0.020796874538064003,
 -0.003313422901555896,
 0.005856591276824474,
 -0.002607909496873617,
 0.016719838604331017,
 0.006288199685513973,
 0.010079711675643921,
 -0.01751665212213993,
 -0.03166012093424797,
 -0.014090348035097122,
 0.017583053559064865,
 -0.0083532789722085,
 0.030093051493167877,
 -0.022231142967939377,
 0.03853929415345192,
 0.00978090614080429,
 -0.00772910704836249

In [20]:
pcds.save_to_disk('./data/')

Saving the dataset (1/1 shards): 100%|██████████| 13139/13139 [00:00<00:00, 85024.22 examples/s]


In [6]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct

qdrant_client = QdrantClient(
    url="https://b8f17857-c547-4c58-8ba8-b479a15dacad.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key=QDRANT_API_KEY,
)

In [21]:
qdrant_client.recreate_collection(collection_name="pokemon-cards", 
                                  vectors_config=models.VectorParams(size=len(emb),
                                                                     distance=models.Distance.COSINE
                                                                     ))

True

In [22]:
import torch

In [23]:
from time import sleep

In [60]:
# # upload records into qdrant
# from tqdm import tqdm, trange
# for batch in tqdm(dl):
#     texts = batch['text']
#     ids = batch['_id']

#     check = qdrant_client.retrieve(collection_name="pokemon-cards", ids=[ids[0].item()])

#     if check != []:
#         # print(check)
#         continue

#     resps = openai.Embedding.create(model="text-embedding-ada-002",
#                                     input=texts)
#     embs = [x['embedding'] for x in resps['data']]

#     blist = []
#     for i in range(len(texts)):
#         sample = {}
#         for key in batch.keys():
#             if type(batch[key][i]) == torch.Tensor:
#                 sample[key] = batch[key][i].item()
#             else:
#                 sample[key] = batch[key][i]
#         blist.append(sample)

#     records = [PointStruct(id=sample['_id'], vector=emb, payload=sample) for (emb, sample) in zip(embs, blist)]
#     qdrant_client.upsert(collection_name='pokemon-cards', wait=True, points=records)
#     sleep(1)
#     # break

 72%|███████▏  | 1189/1643 [09:39<03:41,  2.05it/s]


ResponseHandlingException: The read operation timed out

In [32]:
points = []
for obj in tqdm(pcds):
    emb = obj['embedding']
    id = obj['_id']

    check = qdrant_client.retrieve(collection_name="pokemon-cards", ids=[id])
    if check != []:
        continue

    payload = { k:v for (k, v) in obj.items() if k not in [ 'embedding', '_id' ] }

    ps = PointStruct(id = id, vector=emb, payload=payload)
    points.append(ps)

100%|██████████| 13139/13139 [1:02:27<00:00,  3.51it/s]


In [33]:
len(points)

12791

In [40]:
for p in tqdm(points):
    qdrant_client.upsert(collection_name='pokemon-cards', wait=True, points=[p])

  0%|          | 0/12791 [00:00<?, ?it/s]

100%|██████████| 12791/12791 [1:05:42<00:00,  3.24it/s]


In [41]:
for obj in tqdm(pcds):
    emb = obj['embedding']
    id = obj['_id']

    check = qdrant_client.retrieve(collection_name="pokemon-cards", ids=[id])
    if check != []:
        continue
    else:
        print(id)

100%|██████████| 13139/13139 [1:03:59<00:00,  3.42it/s]


In [56]:
qdrant_client.retrieve(collection_name="pokemon-cards", ids=[0])

[Record(id=0, payload={'_id': 0, 'blip_caption': 'a pokemon that looks like a white and blue animal', 'caption': "A Basic, SP Pokemon Card of type Darkness with the title Absol G and 70 HP of rarity Rare Holo from the set Supreme Victors.  It has the attack Feint Attack with the cost Darkness, the energy cost 1 with the description: Choose 1 of your opponent's Pokemon. This attack does 20 damage to that Pokemon. This attack's damage isn't affected by Weakness, Resistance, Poke-Powers, Poke-Bodies, or any other effects on that Pokemon. It has the attack Doom News with the cost Darkness, Colorless, Colorless, the energy cost 3 with the description: Return all Energy cards attached to Absol G to your hand. The Defending Pokemon is Knocked Out at the end of your opponent's next turn. It has weakness against Fighting 2. It has resistance against Psychic -20. ", 'card_image': 'https://images.pokemontcg.io/pl3/1_hires.png', 'hp': 70, 'id': 'pl3-1', 'length': 873, 'name': 'Absol G', 'pokedex_t

In [18]:
batch = next(iter(dl))
texts = batch['text']
print(f"Batch length is: {len(texts)}")

resps = openai.Embedding.create(model="text-embedding-ada-002",
                                    input=texts)
embs = [x['embedding'] for x in resps['data']]
print(f"Embeddings passed back: {len(embs)}")

blist = []
for i in range(len(batch['text'])):
    sample = {}
    for key in batch.keys():
        if type(batch[key][i]) == torch.Tensor:
            sample[key] = batch[key][i].item()
        else:
            sample[key] = batch[key][i]
    blist.append(sample)

print(f"Number of items in blist: {len(blist)}")

Batch length is: 8
Embeddings passed back: 8
Number of items in blist: 8


In [38]:
resp = openai.Embedding.create(model="text-embedding-ada-002", input="charizard")
emb = resp['data'][0]['embedding']
print(len(emb))

1536


In [42]:
hits = qdrant_client.search(
    collection_name="pokemon-cards",
    query_vector=emb,
    limit=5,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'blip_caption': 'a pokemon that looks like a white ball with a pink nose', 'caption': 'A Basic, V Pokemon Card of type Grass with the title Hisuian Electrode V and 210 HP of rarity Promo from the set SWSH Black Star Promos.  It has the attack Tantrum Blast with the cost [-], the energy cost 1 and the damage of 100 with the description: This attack does 100 damage for each Special Condition affecting this Pokemon. It has the attack Solar Shot with the cost Grass, Colorless, the energy cost 2 and the damage of 120 with the description: Discard all Energy from this Pokemon. It has weakness against Fire 2. ', 'card_image': 'https://images.pokemontcg.io/swshp/SWSH294_hires.png', 'hp': 210, 'id': 'swshp-SWSH294', 'length': 746, 'name': 'Hisuian Electrode V', 'pokedex_text': 'It stores electric energy under very high pressure. It often explodes with little or no provocation. Stores electrical energy inside its body. Even the slightest shock could trigger a huge explosion. It is dangerous. If

In [7]:
qdrant_client.count(collection_name="pokemon-cards")

CountResult(count=13139)

In [8]:
qdrant_client.get_collection(collection_name="pokemon-cards")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=13139, indexed_vectors_count=10001, points_count=13139, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_sch

In [10]:
response = openai.Embedding.create(
  model = "text-embedding-ada-002",
  input = "A fire breathing lizard"
)

emb = response['data'][0]['embedding']
print(len(emb))

1536


In [16]:
search_resp = qdrant_client.search("pokemon-cards", query_vector=emb, limit=10)
search_resp[0].payload['card_image']

'https://images.pokemontcg.io/pgo/10_hires.png'

In [17]:
search_resp[0]

ScoredPoint(id=12593, version=12613, score=0.82062215, payload={'blip_caption': 'a pokemon that looks like a dragon with a flame on its back', 'caption': "A Stage 2 Pokemon Card of type Fire with the title Charizard and 170 HP of rarity Rare Holo evolved from Charmeleon from the set Pokemon GO and the flavor text: It spits fire that is hot enough to melt boulders. It may cause forest fires by blowing flames. It has the attack Flare Blitz with the cost Fire, Fire, Colorless, Colorless, the energy cost 4 and the damage of 170 with the description: Discard all Fire Energy from this Pokemon. It has the ability Burn Brightly with the description: Each basic Fire Energy attached to your Pokemon provides FireFire Energy. You can't apply more than 1 Burn Brightly Ability at a time. It has weakness against Water 2. ", 'card_image': 'https://images.pokemontcg.io/pgo/10_hires.png', 'hp': 170, 'id': 'pgo-10', 'length': 973, 'name': 'Charizard', 'pokedex_text': 'Spits fire that is hot enough to mel